In [31]:
#Import library
import pandas as pd
import numpy as np
import collections
import os
import re
import math

In [ ]:
#Import data 
raw = pd.read_csv('Keywords.csv', encoding='Latin-1')
raw.pop('Unnamed: 0')               
df = raw.replace(np.nan, '', regex=True)
df

In [25]:
# Method of getting the most frequent keywords from an entity
def highlightedTopic(text, entities):
  filter = df[df[entities] == text]
  train = list(filter.Keywords)
  t = [item.split(',') for item in train]
  flat_list = []
  for sublist in t:
    for item in sublist:
      if (item != '') :
        flat_list.append(item)
  counter = collections.Counter(flat_list)
  c= counter.most_common()
  return c[:10]                                     #result is in a form of (Keywords,Frequency)

#For example
#highlightedTopic('Hazelcast','Inc')                #within a company
#highlightedTopic('NVIDIA GTC 2020','Conference')   #within a conference

In [32]:
# Getting the links on Wikipedia and generating common link ratio formula;
# The code is a refernece from .... and is adjusted at some part to synchronize with the current project 

class PageManager(object):
    """Class for managing pages."""

    def __init__(self):
        self.pages = {}

    def load_with_title(self, title):
        """Load the page with given title."""

        if title in self.pages.keys():
            return
        url = "http://en.wikipedia.org/wiki/" + title
        self.pages[title] = Page(url)

    def sort_page_relevance(self, title):
        """Return a sorted list of titles and relevance sorted by relevance
        Relevance is determined by the ratio of common links from both pages
        to total number of links in the second."""
        self.load_with_title(title)
        relevance = []
        page = self.pages[title]
        for t in self.pages:
            ratio = page.get_common_link_ratio(self.pages[t])
            relevance.append((t, ratio))
        relevance.sort(key= lambda x: x[1])
        relevance.reverse()
        return relevance



class Page(object):

    def __init__(self, url):
        self.url = url
        # http://en.wikipedia.org/wiki/TITLE#INNER_LINK
        self.title = url.split("/")[-1].split("#")[0]
        self.title = self.title.replace('%27',"'")
        self.read_page()
        self.get_main_div()
        self.find_links()

    def read_page(self):
        """Read the page
        Tries to read from local disk first. If the file is not found,
        it is downloaded."""
        try:
            tempfile = open(self.title)
        except FileNotFoundError:
            os.system("wget '{}'".format(self.url))
            tempfile = open(self.title)
        self.lines = tempfile.readlines()
        tempfile.close()

    def get_main_div(self):
        self.main_div_content = []
        started = False
        for line in self.lines:
            if not started:
                if line.strip().startswith('<div id="bodyContent'):
                    self.main_div_content.append(line)
                    started = True
            else:
                if line.strip().startswith('<div id="mw-navigation'):
                    break
                self.main_div_content.append(line)
        # Rip off first generic lines (links to search etc.)
        self.main_div_content = self.main_div_content[9:]

    def find_links(self):
        self.links = []
        for line in self.main_div_content:
            match = re.search('/wiki/([a-zA-Z_:]*)', line)
            if not match: continue
            if ":" not in match.group(1): self.links.append(match.group(1))
        self.links = set(self.links)

    def find_common_links(self, page):
        common_links =[]
        for link in self.links:
            if link in page.links:
                common_links.append(link)
        return common_links

    def count_common_links(self, page):
        return len(self.find_common_links(page))

    def get_common_link_ratio(self, page):
        count = self.count_common_links(page)
        return math.tan(((count/(len(page.links)+len(self.links)))*math.pi/2))

In [ ]:
train = list(df.Keywords)
t = [item.split(',') for item in train]
flat_list = []
for sublist in t:
    for item in sublist:
      if (item != '') :
        flat_list.append(item)
        print(item)

In [56]:
b = [item.replace(' ', '_') for item in flat_list]
c = [item.replace('\x96','–')for item in b]
a = [item.replace("'",'%27')for item in c]
a.remove('TofaÅ\x9f')
a.remove('Saffirâ\x80\x93Simpson_scale')
a.remove('1-Trichloroethane')
a.remove('Metropolisâ\x80\x93Hastings_algorithm')

In [57]:
page_manager = PageManager()
example_titles = a

for title in example_titles:
  page_manager.load_with_title(title)
  if (len(Page(title).links)==0):
   print(title)

Average_selling_price
Business_domain


In [58]:
df['Keywords'] = df['Keywords'].apply(lambda x: ''.join([item.replace(' ', '_') for item in x]))
df['Keywords'] = df['Keywords'].apply(lambda x: ''.join([item.replace('\x96','–')for item in x]))
df['Keywords'] = df['Keywords'].apply(lambda x: ''.join([item.replace("'",'%27')for item in x]))
df

,Conference,Inc,Title,Keywords
0,NVIDIA GTC 2020,University of Cambridge,Accelerating Unsupervised SAR Polarimetric Ima...,"Brand,Image_segmentation,List_of_sensors,Synth..."
1,NVIDIA GTC 2020,Hewlett Packard Enterprise,AI The Network Edge (Presented by HPE) [S22480],"Edge_case,Extreme_environment,Process_capabili..."
2,NVIDIA GTC 2020,Teradata,Anomaly Detection on Aircraft Sensor Data Usin...,"Neural_network,Proximity_sensor,Rheology,Senso..."
3,NVIDIA GTC 2020,Altair Engineering,Complex 60 GPU CFD Simulations for Aerospace G...,"Computational_fluid_dynamics,DC-to-DC_converte..."
4,NVIDIA GTC 2020,KAIST,Deep Learning-Based Anti-Drone System [P21900],"Deep_Instinct,Deep_learning_super_sampling"
...,...,...,...,...
399,Oracle 2020,and Central and Eastern Europe,Unlock Endless Possibilities with Oracle Cloud,"Garbage_Pail_Kids,Machine_learning,Oracle_Clou..."
400,Oracle 2020,Oracle,"Cloud Platform, Middleware Strategy and Roadmap","Google_Cloud_Platform,Oracle_Cloud,Oracle_Data..."
401,Oracle 2020,Oracle,A New Class of Cloud Infrastructure,Oracle_Team_USA
402,Oracle 2020,Oracle,Solution Keynote: Strategy and Roadmap for Clo...,"Online_transaction_processing,Oracle_Linux"


In [60]:
def wikiCompare(list1,list2):
  common_ratio = np.zeros(len(list1)*len(list2))
  for i in range(0,len(list1)):
    for j in range(0,len(list2)):
      common_ratio[i*len(list2)+j] = page_manager.pages[list1[i]].get_common_link_ratio(page_manager.pages[list2[j]])
  return np.average(common_ratio) 

In [64]:
FullKey = df[df['Keywords'] != '']
e = list(FullKey.Inc.unique())
len(e)
initializeMatrix = np.zeros((len(e), len(e)))
initializeMatrix.shape
for i in range(0,len(e)):
  for j in range(0,len(e)):
    comp1 = FullKey[FullKey['Inc'] == e[i]]
    comp2 = FullKey[FullKey['Inc'] == e[j]]
    list1 = [item.split(',') for item in comp1['Keywords'].values]
    list2 = [item.split(',') for item in comp2['Keywords'].values]

    stopwords =['You_didn%27t_build_that','Average_selling_price','Business_domain','TofaÅ\x9f','Saffirâ\x80\x93Simpson_scale','1-Trichloroethane','Metropolisâ\x80\x93Hastings_algorithm']

    flatten1 = []
    for sublist in list1:
      for item in sublist:
        if item not in stopwords:
          flatten1.append(item)

    flatten2 = [] 
    for sublist in list2:
      for item in sublist:
        if item not in stopwords:
          flatten2.append(item)

    initializeMatrix[i,j] = wikiCompare(list1=flatten1, list2=flatten2)
initializeMatrix

array([[0.25730223, 0.00958185, 0.02479069, ..., 0.00617866, 0.01140707,
        0.00722927],
       [0.00958185, 0.26735736, 0.01344889, ..., 0.00567682, 0.01126335,
        0.00377722],
       [0.02479069, 0.01344889, 0.27048784, ..., 0.0088209 , 0.01439564,
        0.0098338 ],
       ...,
       [0.00617866, 0.00567682, 0.0088209 , ..., 0.2750382 , 0.04523763,
        0.01857568],
       [0.01140707, 0.01126335, 0.01439564, ..., 0.04523763, 0.21006609,
        0.02538182],
       [0.00722927, 0.00377722, 0.0098338 , ..., 0.01857568, 0.02538182,
        0.25839111]])

In [66]:
final = pd.DataFrame(initializeMatrix, e, e)
final

,University of Cambridge,Hewlett Packard Enterprise,Teradata,Altair Engineering,KAIST,Nasa Langley Research Center,Ridge-i inc.,The Boeing Company,Centauri,SURVICE Engineering,Missouri University of Science and Technology,Virginia Tech,Davis,Pacific Northwest National Laboratory,Lawrence Berkeley National Laboratory,Oak Ridge National Laboratory,Georgia Tech,NVIDIA,University of Texas at Austin,Japan Agency for Marine-Earth Science and Technology,Helmholtz-Zentrum Dresden-Rossendorf,University of Illinois,Los Alamos National Laboratory,NERSC,Japan Atomic Energy Agency,Sandia National Laboratories,The National Energy Technology Laboratory,California Institute of Technology,NIST,Brazilian Synchrotron Light Laboratory / CNPEM,NIKHEF and University of Maastricht,University of Tennessee,Barcelona Supercomputing Center,National Center for Atmospheric Research,Naval Research Lab,Aselsan Research Center,Predictive Science Inc.,RIKEN Center for Computational Science,DST/CSIRO,JÃ¼lich Supercomputing Centre,...,Fundbox,Hybrid Theory,Digital Fingerprints,"AI Product Institute, Silicon Valley",MIT Sloan School of Management,"Dell Technologies, Vmware",AI in Business,Wells Fargo,Ideal State,SMG / CMSWire,Hootsuite,Always Day One,GE Healthcare,Nexthink,VMware,Gannett,IEEE ICICLE,Reworked / CMSWire,Fast Company,Workgrid,Reworked/CMSWire,Shopify,EIS,Simpplr,"RiverStone, Workgrid",LumApps,Sinequa,StitchDX,Environmental Science Associates,Optimal Disruption,Oracle,Oracle Cloud Infrastructure,Deloitte,Oracle Cloud,and Africa,PwC,Asia Pacific and Japan,Oracl,Oracle Applications Product Development,and Central and Eastern Europe
University of Cambridge,0.257302,0.009582,0.024791,0.015043,0.008575,0.010205,0.020092,0.013562,0.012443,0.009626,0.005818,0.015876,0.018279,0.010741,0.006413,0.022600,0.006750,0.014228,0.011422,0.015685,0.005167,0.007401,0.013095,0.003536,0.010003,0.003519,0.008431,0.010909,0.015720,0.042179,0.008356,0.010357,0.006761,0.012572,0.008368,0.007839,0.006798,0.003539,0.009101,0.008388,...,0.003446,0.009881,0.010246,0.004221,0.010697,0.003058,0.005638,0.006521,0.009087,0.016823,0.010442,0.004445,0.0,0.009290,0.002919,0.005678,0.014236,0.004956,0.001158,0.000000,0.011923,0.021942,0.005648,0.005746,0.012923,0.007335,0.005638,0.005638,0.021942,0.006377,0.007148,0.007105,0.005722,0.011669,0.008105,0.021211,0.011170,0.006179,0.011407,0.007229
Hewlett Packard Enterprise,0.009582,0.267357,0.013449,0.010599,0.005065,0.012809,0.010950,0.017971,0.011771,0.011809,0.003021,0.010159,0.016656,0.011135,0.004175,0.012955,0.005350,0.009114,0.007285,0.010048,0.003966,0.006576,0.016093,0.001300,0.004636,0.001275,0.000000,0.007674,0.011351,0.034536,0.010694,0.005085,0.007406,0.006980,0.003390,0.006925,0.004586,0.008358,0.003927,0.006682,...,0.000000,0.007472,0.002684,0.008515,0.002827,0.001743,0.006882,0.005090,0.011449,0.009776,0.012545,0.005077,0.0,0.003999,0.005614,0.001676,0.008411,0.003800,0.000000,0.000000,0.003661,0.018613,0.004284,0.001909,0.012319,0.005897,0.006882,0.006882,0.018613,0.004757,0.005121,0.005954,0.000000,0.007227,0.003283,0.016981,0.007918,0.005677,0.011263,0.003777
Teradata,0.024791,0.013449,0.270488,0.023254,0.011727,0.012001,0.019840,0.023170,0.016613,0.014719,0.007246,0.016072,0.027177,0.010493,0.007353,0.025981,0.010419,0.024334,0.014156,0.080838,0.006322,0.012204,0.023581,0.004318,0.007717,0.002647,0.003314,0.013112,0.018046,0.040336,0.010154,0.009674,0.008594,0.014041,0.010744,0.007866,0.006758,0.002785,0.010698,0.009621,...,0.001465,0.019530,0.005926,0.009837,0.008676,0.002027,0.007475,0.007731,0.012050,0.020139,0.010229,0.007308,0.0,0.009124,0.008262,0.006153,0.014711,0.005331,0.000000,0.000000,0.011309,0.029753,0.008484,0.003989,0.019781,0.007860,0.007475,0.007475,0.029753,0.008131,0.008648,0.010362,0.002766,0.013801,0.007478,0.024933,0.013522,0.008821,0.014396,0.009834
Altair Engineering,0.015043,0.010599,0.023254,0.352566,0.008126,0.017668,0.022301,0.024276,0.011996,0.018722,0.006875,0.019545,0.014586,0.012184,0.0